In [9]:
import os
import torch
from torch import nn
import torch.nn.functional as F
from collections import OrderedDict

import timm
from timm.data.transforms_factory import create_transform
from timm.data import resolve_data_config

In [32]:
CP_PATH = "weights/save_pneumonia_imagnet_resnet50"

In [33]:
ckpt = torch.load(CP_PATH + "/model_timm_final.pth")

### Reformat dict keys

In [34]:
new_ckpt = OrderedDict()
for key in ckpt.keys():
    new_key = key[6:]
    new_ckpt[new_key] = ckpt[key]

In [38]:
torch.save(new_ckpt, CP_PATH + "/test_ckpt.pth")

### Load model

In [39]:
model = timm.create_model(model_name="resnet50", checkpoint_path = CP_PATH + "/test_ckpt.pth")

### Test on data

In [71]:
img_test_path = "data/chest_xray/test/PNEUMONIA/person1_virus_6.jpeg"
img = Image.open(img_test_path).convert("RGB")

In [72]:
config = resolve_data_config({}, model=model)
transform = create_transform(**config)

In [86]:
tensor = transform(img).unsqueeze(0)

In [87]:
with torch.no_grad():
    out = model(tensor)

proba = torch.nn.functional.softmax(out[0],dim=0)
print(proba[:2])

tensor([0.0276, 0.9717])
